In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("small_movie_reviews.csv")

/tmp/ipykernel_1550/3904227517.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("small_movie_reviews.csv")


In [3]:
df.head()

,Unnamed: 0,rating,review_title,text,images_x,asin,parent_asin,user_id,timestamp,helpful_vote,...,features,description,price,images_y,videos,store,categories,details,bought_together,author
0,0,5.0,Five Stars,"Amazon, please buy the show! I'm hooked!",[],B013488XFS,B013488XFS,AGGZ357AO26RQZVRLGU4D4N52DZQ,1440385637000,0,...,"['IMDb 8.1', '2017', '10 episodes', 'X-Ray', '...",['A\xa0con man (Giovanni Ribisi) on the run fr...,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,"['Suspense', 'Drama']","{'Content advisory': ['Nudity', 'violence', 's...",NaN,NaN
1,10,5.0,Five Stars,LOVE this movie! Hits you in the feels!,[],B01LWY8995,B01LWY8995,AGXVBIUFLFGMVLATYXHJYL4A5Q7Q,1485870227000,0,...,NaN,NaN,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,NaN,"{'Content advisory': ['Violence', 'alcohol use...",NaN,NaN
2,20,5.0,Beautiful Move for the Entire Familia!,We loved Coco! My husband (Mexican) and I (Cau...,[],B0779KMWG1,B0779KMWG1,AHTTU2FL6FCNBBAESCJHOHHSSW7A,1513539059497,7,...,NaN,NaN,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,NaN,{'Audio languages': ['English Dialogue Boost: ...,NaN,NaN
3,30,5.0,Five Stars,VERY GOOD MOVIE: WAS LIKED BY ALL WHO SEEN IT,[],B001QTXM5Y,B001QTXM5Y,AEFKF6R2GUSK2AWPSWRR4ZO36JVQ,1414899539000,0,...,[],"['Product Description', 'Watchmen: Special Edi...",12.40,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Watchmen home-video trailer', 'url...","Malin Akerman (Actor), Billy Crudup (Act...","['Movies & TV', 'Genre for Featured Categories...","{'Genre': 'Science Fiction & Fantasy, Drama, A...",NaN,NaN
4,40,5.0,FIFTH ELEMENT ultimate edition,one of my classics: This one I and friends can...,[],B0006GVJE4,B0006GVJE4,AEFKF6R2GUSK2AWPSWRR4ZO36JVQ,1364088328000,0,...,[],"['Product Description', ""New York cab driver K...",19.12,[{'thumb': 'https://m.media-amazon.com/images/...,[],"Bruce Willis (Actor), Gary Oldman (Actor...","['Movies & TV', 'Science Fiction & Fantasy', '...","{'Aspect Ratio': '2.35:1', 'Is Discontinued By...",NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         100000 non-null  int64  
 1   rating             100000 non-null  float64
 2   review_title       99990 non-null   object 
 3   text               99993 non-null   object 
 4   images_x           100000 non-null  object 
 5   asin               100000 non-null  object 
 6   parent_asin        100000 non-null  object 
 7   user_id            100000 non-null  object 
 8   timestamp          100000 non-null  int64  
 9   helpful_vote       100000 non-null  int64  
 10  verified_purchase  100000 non-null  bool   
 11  main_category      99082 non-null   object 
 12  movie_title        45657 non-null   object 
 13  subtitle           1997 non-null    object 
 14  average_rating     99999 non-null   float64
 15  rating_number      99999 non-null   float64
 16  fea

In [5]:
from data_pipeline import preprocess, encode

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df["text"] = df["text"].fillna("")

In [7]:


# Preprocess the corpus
vocab, tokens = preprocess(df["text"])

# Encode the preprocessed tokens using the specified encoding method
encoding_method = 'Bag-of-Words'
encoded_tokens = encode(tokens, encoding_method)

In [8]:
encoded_tokens

<100000x65815 sparse matrix of type '<class 'numpy.int64'>'
	with 3130010 stored elements in Compressed Sparse Row format>

In [9]:
encoding_method = 'TF-IDF'
encoded_tokens = encode(tokens, encoding_method)

In [10]:
encoded_tokens

<100000x65815 sparse matrix of type '<class 'numpy.float64'>'
	with 3130010 stored elements in Compressed Sparse Row format>

In [11]:
encoding_method = 'Word2Vec'
encoded_tokens = encode(tokens, encoding_method)

In [12]:
encoded_tokens

array([[ 1.90208226e-01,  2.08543412e+00,  2.62253604e+00, ...,
         4.89238155e+00,  2.45440781e+00, -1.44998643e+00],
       [-4.83833818e+00,  1.22370816e+00, -4.11136732e+00, ...,
        -2.38931644e+00,  2.12980786e+00, -4.58378200e+00],
       [-3.77459506e+01,  3.43863683e+01, -7.80979048e+00, ...,
        -1.36801216e+01,  2.86219738e+01,  1.50732197e+01],
       ...,
       [-6.96688709e+00,  1.06795651e+01,  6.20685539e+00, ...,
        -6.23384338e+00,  6.56391585e+00,  2.04785072e+00],
       [ 1.15907937e-03, -4.07711484e+00, -5.92606105e+00, ...,
        -5.79564766e+00,  1.61338625e+01, -8.56407300e-01],
       [-4.85056081e+00,  5.18127815e+00, -5.04542007e+00, ...,
        -4.80608353e+00,  1.00689037e+01,  5.06252014e+00]])